players__dataset_creation code for past seasons

Select the season in season variable, and repeat the code if needed

In [1]:
season = '2324'

In [2]:
import pandas as pd

In [3]:
rcsv = pd.read_csv('fbref_data/season' + season + '/outfield_players.csv')   
outfield_players = pd.DataFrame(rcsv)

rcsv = pd.read_csv('fbref_data/season' + season + '/keepers_players.csv')   
keeper_players = pd.DataFrame(rcsv)

In [4]:
players = pd.concat(   [ outfield_players[['player', 'team']], keeper_players[['player', 'team']] ], axis = 0, ignore_index = True)
keepers_ID = len(outfield_players)
players

,player,team
0,Tammy Abraham,Roma
1,Francesco Acerbi,Inter
2,Yacine Adli,Milan
3,Michel Aebischer,Bologna
4,Lucien Agoume,Inter
...,...,...
662,Marco Sportiello,Milan
663,Mile Svilar,Roma
664,Wojciech Szczęsny,Juventus
665,Pietro Terracciano,Fiorentina


In [5]:
import unicodedata

def normalize_name(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf-8')

In [6]:
players['surname'] = players['player']
players['initial'] = players['player']

for i in range(players.shape[0]):
    players['surname'][i] = players['surname'][i].split(' ')[-1]
    players['surname'][i] = normalize_name(players['surname'][i]).replace('\'', '')
    
    
    players['initial'][i] = players['player'][i][0]

In [7]:
print(players[['player', 'surname']].to_string())

                         player           surname
0                 Tammy Abraham           Abraham
1              Francesco Acerbi            Acerbi
2                   Yacine Adli              Adli
3              Michel Aebischer         Aebischer
4                 Lucien Agoume            Agoume
5                    Marley Aké               Ake
6          Ebenezer Akinsanmiro       Akinsanmiro
7        Jean-Daniel Akpa-Akpro        Akpa-Akpro
8            David Akpan Ankeye            Ankeye
9                  Luis Alberto           Alberto
10               Carlos Alcaraz           Alcaraz
11              Pontus Almqvist          Almqvist
12             Lorenzo Amatucci          Amatucci
13                 Bruno Amione            Amione
14              Felipe Anderson          Anderson
15                     Angeliño          Angelino
16                Houssem Aouar             Aouar
17                Simone Aresti            Aresti
18             Marko Arnautović        Arnautovic


In [8]:
rcsv = pd.read_csv('config/name_fix.txt')   
name_fix = pd.DataFrame(rcsv)

for i in range(name_fix.shape[0]):
    for j in range(players.shape[0]):
        if(players['surname'][j].lower() == name_fix['FROM'][i].lower() and players['team'][j].lower() == name_fix['TEAM'][i].lower()):
            players['surname'][j] = name_fix['TO'][i]
            print(name_fix['TO'][i])

name_fix


Ostigard
Kjaer
Djuric
Gudmundsson
Nissen


,FROM,TO,TEAM
0,stigard,Ostigard,Napoli
1,Min-jae,Kim,Napoli
2,Hjlund,Hojlund,Atalanta
3,Gytkjr,Gytkjaer,Monza
4,Carlos,Augusto,Inter
5,Mhle,Maehle,Atalanta
6,Kjr,Kjaer,Milan
7,uricic,Djuricic,Sampdoria
8,uric,Djuric,Hellas Verona
9,Arthur,Cabral,Fiorentina


In [9]:
fc_data = pd.read_excel('fantacalcio/season' + season + '/Quotazioni_Fantacalcio.xlsx', 'Tutti', header = 1)

fc_players = fc_data [['Id', 'R', 'Nome', 'Squadra']]

fc_players = fc_players.rename(columns = {'Id' : 'id', 'R': 'r', 'Nome' : 'name', 'Squadra' : 'team'})

fc_players['surname'] = fc_players['name']
fc_players['initial'] = fc_players['name']


for i in range(fc_players.shape[0]):
    spl = normalize_name( fc_players['name'][i].replace('\'', '') ).split(' ')
    if('.' in spl[-1]):
        fc_players.loc[i, 'surname'] = spl[-2]
        fc_players.loc[i, 'initial'] = spl[-1][0]
    else:
        fc_players.loc[i, 'surname'] = spl[-1]
        fc_players.loc[i, 'initial'] = ''
    
fc_players



,id,r,name,team,surname,initial
0,2428,P,Sommer,Inter,Sommer,
1,133,P,Skorupski,Bologna,Skorupski,
2,5876,P,Di Gregorio,Monza,Gregorio,
3,572,P,Meret,Napoli,Meret,
4,2814,P,Provedel,Lazio,Provedel,
...,...,...,...,...,...,...
537,6160,A,Vivaldo,Udinese,Vivaldo,
538,6242,A,Bidaoui,Frosinone,Bidaoui,
539,6418,A,Burnete,Lecce,Burnete,
540,6419,A,Corfitzen,Lecce,Corfitzen,


In [10]:
fc_players['fb_ID'] = fc_players['id']

for i in range(fc_players.shape[0]):
    fc_players.loc[i, 'fb_ID']  = -1
    
    for j in range(players.shape[0]):
        if(fc_players['team'][i].lower() in players['team'][j].lower()):
            if(fc_players['surname'][i].lower() == players['surname'][j].lower()):              
                # if(fc_players['initial'][i] == '' or fc_players['initial'][i].lower() == players['initial'][j].lower()):
                if((fc_players['r'][i] == 'P') == (j >= keepers_ID)): # check wether they're a goalkeeper for both FBREF and Fantacalcio
                    fc_players.loc[i, 'fb_ID'] = j

In [11]:
for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] == -1):
        print(fc_players['name'][i])

Sepe
Lamanna
Pegolo
Padelli
Frattali
Contini
Brancolini
Berardi A.
Boer
Popa
Stubljar
Gori
Borbei
Kristensen
Tressoldi
Gunter
Soumaoro
Dermaku
Tonelli
Capradossi
Kumbulla
Amey
Guarino
Smajlovic
N'guessan
Kalaj
Pierozzi
Lindstrom
Iling Junior
Cajuste 
Machin
Basic
Legowski
Rog
Jagiello
Akpa Akpro
Hrustic
Faticanti
Belardinelli
Camara E.
Joselito
Deulofeu
Jovane
Alvarez A.
Yildiz
Braaf
Vivaldo
Bidaoui


In [12]:
fc_players

,id,r,name,team,surname,initial,fb_ID
0,2428,P,Sommer,Inter,Sommer,,660
1,133,P,Skorupski,Bologna,Skorupski,,658
2,5876,P,Di Gregorio,Monza,Gregorio,,629
3,572,P,Meret,Napoli,Meret,,639
4,2814,P,Provedel,Lazio,Provedel,,652
...,...,...,...,...,...,...,...
537,6160,A,Vivaldo,Udinese,Vivaldo,,-1
538,6242,A,Bidaoui,Frosinone,Bidaoui,,-1
539,6418,A,Burnete,Lecce,Burnete,,87
540,6419,A,Corfitzen,Lecce,Corfitzen,,137


In [13]:
#Data for Outfield players
columns_to_copy = outfield_players.columns[4:]

fc_players[columns_to_copy] = 0

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] != 'P'):
         for j in range(columns_to_copy.shape[0]):
            #fc_players[columns_to_copy[j]][i] = outfield_players[columns_to_copy[j]][fc_players['fb_ID'][i]]
            fc_players.loc[i, columns_to_copy[j]] = outfield_players.loc[fc_players['fb_ID'][i], columns_to_copy[j]]
            

C:\Users\mbolo\AppData\Local\Temp\ipykernel_10576\2053436513.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy] = 0
C:\Users\mbolo\AppData\Local\Temp\ipykernel_10576\2053436513.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy] = 0
C:\Users\mbolo\AppData\Local\Temp\ipykernel_10576\2053436513.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns a

In [14]:
keeper_players.columns[4:]

Index(['age', 'birth_year', 'gk_games', 'gk_games_starts', 'gk_minutes',
       'gk_goals_against', 'gk_goals_against_per90',
       'gk_shots_on_target_against', 'gk_saves', 'gk_save_pct', 'gk_wins',
       'gk_ties', 'gk_losses', 'gk_clean_sheets', 'gk_clean_sheets_pct',
       'gk_pens_att', 'gk_pens_allowed', 'gk_pens_saved', 'gk_pens_missed',
       'minutes_90s', 'gk_free_kick_goals_against',
       'gk_corner_kick_goals_against', 'gk_own_goals_against', 'gk_psxg',
       'gk_psnpxg_per_shot_on_target_against', 'gk_psxg_net',
       'gk_psxg_net_per90', 'gk_passes_completed_launched',
       'gk_passes_launched', 'gk_passes_pct_launched', 'gk_passes',
       'gk_passes_throws', 'gk_pct_passes_launched', 'gk_passes_length_avg',
       'gk_goal_kicks', 'gk_pct_goal_kicks_launched',
       'gk_goal_kick_length_avg', 'gk_crosses', 'gk_crosses_stopped',
       'gk_crosses_stopped_pct', 'gk_def_actions_outside_pen_area',
       'gk_def_actions_outside_pen_area_per90', 'gk_avg_distance_

In [15]:
#Data for Keepers
columns_to_copy = keeper_players.columns[4:]

fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year

delta_k = outfield_players.shape[0]

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] == 'P'):
        if(fc_players['fb_ID'][i] - delta_k >= 0):
            for j in range(columns_to_copy.shape[0]):          
                #fc_players[columns_to_copy[j]][i] = keeper_players[columns_to_copy[j]][fc_players['fb_ID'][i] - delta_k]
                fc_players.loc[i, columns_to_copy[j]] = keeper_players.loc[fc_players['fb_ID'][i] - delta_k, columns_to_copy[j]]

C:\Users\mbolo\AppData\Local\Temp\ipykernel_10576\621407790.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year
C:\Users\mbolo\AppData\Local\Temp\ipykernel_10576\621407790.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year
C:\Users\mbolo\AppData\Local\Temp\ipykernel_10576\621407790.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usual

In [16]:
keeper_players

,player,nationality,position,team,age,birth_year,gk_games,gk_games_starts,gk_minutes,gk_goals_against,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,Simone Aresti,it ITA,GK,Cagliari,37,1986,1.0,0.0,1.0,1.0,...,64.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.00,0.0
1,Emil Audero,it ITA,GK,Inter,26,1997,4.0,4.0,337.0,3.0,...,24.6,18.0,16.7,27.2,54.0,6.0,11.1,2.0,0.53,10.2
2,Nicola Bagnolini,it ITA,GK,Bologna,19,2004,1.0,0.0,5.0,0.0,...,47.0,2.0,50.0,53.0,0.0,0.0,0.0,0.0,0.00,0.0
3,Etrit Berisha,al ALB,GK,Empoli,34,1989,14.0,14.0,1260.0,25.0,...,33.0,90.0,50.0,40.8,236.0,11.0,4.7,10.0,0.71,12.4
4,Elia Caprile,it ITA,GK,Empoli,21,2001,23.0,23.0,2070.0,27.0,...,36.8,159.0,53.5,45.1,391.0,31.0,7.9,20.0,0.87,13.3
5,Marco Carnesecchi,it ITA,GK,Atalanta,23,2000,27.0,27.0,2425.0,30.0,...,33.8,193.0,67.4,51.7,332.0,20.0,6.0,39.0,1.45,14.7
6,Michele Cerofolini,it ITA,GK,Frosinone,24,1999,8.0,7.0,675.0,11.0,...,37.2,38.0,50.0,42.8,126.0,4.0,3.2,7.0,0.93,14.3
7,Oliver Christensen,dk DEN,GK,Fiorentina,24,1999,4.0,4.0,360.0,9.0,...,31.0,20.0,35.0,33.7,32.0,1.0,3.1,9.0,2.25,19.5
8,Andrea Consigli,it ITA,GK,Sassuolo,36,1987,35.0,35.0,3150.0,68.0,...,36.9,252.0,49.6,41.2,545.0,23.0,4.2,22.0,0.63,11.8
9,Nikita Contini Baranovsky,it ITA,GK,Napoli,27,1996,1.0,0.0,17.0,0.0,...,17.0,1.0,0.0,15.0,1.0,0.0,0.0,3.0,15.88,37.0


In [17]:
fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,2428,P,Sommer,Inter,Sommer,,660,34,1988,0,...,26.1,201,22.9,27.9,395,18,4.6,14,0.41,12.1
1,133,P,Skorupski,Bologna,Skorupski,,658,32,1991,0,...,29.6,188,42.0,38.7,432,12,2.8,15,0.47,12.2
2,5876,P,Di Gregorio,Monza,Gregorio,,629,26,1997,0,...,30.6,223,52.5,44.2,469,19,4.1,18,0.56,11.5
3,572,P,Meret,Napoli,Meret,,639,26,1997,0,...,27.9,137,22.6,29.5,334,9,2.7,29,0.94,15.4
4,2814,P,Provedel,Lazio,Provedel,,652,29,1994,0,...,30.8,176,35.2,37.0,377,15,4.0,18,0.60,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,6160,A,Vivaldo,Udinese,Vivaldo,,-1,0,0,0,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
538,6242,A,Bidaoui,Frosinone,Bidaoui,,-1,0,0,0,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
539,6418,A,Burnete,Lecce,Burnete,,87,19,2004,1,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
540,6419,A,Corfitzen,Lecce,Corfitzen,,137,18,2004,1,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0


In [18]:
import numpy as np

votes = pd.read_excel('mid_outputs/season' + season + '/players_votes.xlsx', index_col = 0)

mean_def = 6
std_def = 0.58

mean_def_P = 6.22
std_def_P = 0.43


min_votes = 6


perf_df = pd.DataFrame(columns = ['vote_avg', 'vote_std'])

for i in range(fc_players.shape[0]):
    v = np.array([])
    for j in range(votes.shape[0]):
        if(fc_players['name'][i] == votes['player'][j]):
            v = np.append(v, votes['vote'][j])
    
    if(fc_players['r'][i] == 'P'):
        mean_def_i = mean_def_P
        std_def_i = std_def_P
        
    if(v.shape[0] < min_votes):
         for k in range(min_votes - v.shape[0]):
            v = np.append( v, np.random.normal(mean_def_i, std_def_i) )
    
    row_df = pd.DataFrame(data = [[np.mean(v), np.std(v)]], columns = perf_df.columns)
    perf_df = pd.concat([perf_df, row_df], ignore_index = True)
    
perf_df
    

,vote_avg,vote_std
0,6.117647,0.470588
1,6.171875,0.461136
2,6.348485,0.583678
3,6.064516,0.435185
4,6.233333,0.478423
...,...,...
537,6.212581,0.405030
538,6.093504,0.362881
539,6.147232,0.532420
540,6.254964,0.341970


In [19]:
fc_players = pd.concat([fc_players, perf_df], axis = 1)

fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions,vote_avg,vote_std
0,2428,P,Sommer,Inter,Sommer,,660,34,1988,0,...,22.9,27.9,395,18,4.6,14,0.41,12.1,6.117647,0.470588
1,133,P,Skorupski,Bologna,Skorupski,,658,32,1991,0,...,42.0,38.7,432,12,2.8,15,0.47,12.2,6.171875,0.461136
2,5876,P,Di Gregorio,Monza,Gregorio,,629,26,1997,0,...,52.5,44.2,469,19,4.1,18,0.56,11.5,6.348485,0.583678
3,572,P,Meret,Napoli,Meret,,639,26,1997,0,...,22.6,29.5,334,9,2.7,29,0.94,15.4,6.064516,0.435185
4,2814,P,Provedel,Lazio,Provedel,,652,29,1994,0,...,35.2,37.0,377,15,4.0,18,0.60,13.0,6.233333,0.478423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,6160,A,Vivaldo,Udinese,Vivaldo,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.212581,0.405030
538,6242,A,Bidaoui,Frosinone,Bidaoui,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.093504,0.362881
539,6418,A,Burnete,Lecce,Burnete,,87,19,2004,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.147232,0.532420
540,6419,A,Corfitzen,Lecce,Corfitzen,,137,18,2004,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.254964,0.341970


In [23]:
min_gk_games = 6

fc_players_newgk = fc_players.copy()

columns_to_avg = fc_players.columns[123:] # from gk_games to end

for i in range(fc_players.shape[0]):
    if(fc_players['r'][i] == 'P'):
        if(fc_players['gk_games'][i] < min_gk_games):
            for j in range(fc_players.shape[0]):
                if(fc_players['team'][i] == fc_players['team'][j] and fc_players['gk_games'][j] >= min_gk_games):
                    break
                    
            weight = 1 - (min_gk_games - fc_players['gk_games'][i]) / min_gk_games
            
            fc_players_newgk.loc[i][columns_to_avg] = fc_players.loc[i][columns_to_avg] * weight + (1 - weight) * fc_players.loc[j][columns_to_avg]
            
            print(fc_players['name'][i] + ', ' + str(weight))
            
        

Perin, 0.5
Cragno, 0.5
Mirante, 0.33333333333333337
Sepe, 0.0
Leali, 0.5
Lamanna, 0.0
Sommariva, 0.16666666666666663
Pegolo, 0.0
Perilli, 0.16666666666666663
Padelli, 0.0
Perisan, 0.16666666666666663
Audero, 0.6666666666666667
Di Gennaro, 0.16666666666666663
Pinsoglio, 0.16666666666666663
Aresti, 0.16666666666666663
Fiorillo, 0.6666666666666667
Rossi F., 0.16666666666666663
Frattali, 0.0
Contini, 0.0
Brancolini, 0.0
Berardi A., 0.0
Gemello, 0.33333333333333337
Boer, 0.0
Bagnolini, 0.16666666666666663
Martinelli T., 0.16666666666666663
Popa, 0.0
Stubljar, 0.0
Gori, 0.0
Christensen O., 0.6666666666666667
Borbei, 0.0


C:\Users\mbolo\AppData\Local\Temp\ipykernel_10576\3833599976.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players_newgk.loc[i][columns_to_avg] = fc_players.loc[i][columns_to_avg] * weight + (1 - weight) * fc_players.loc[j][columns_to_avg]
C:\Users\mbolo\AppData\Local\Temp\ipykernel_10576\3833599976.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players_newgk.loc[i][columns_to_avg] = fc_players.loc[i][columns_to_avg] * weight + (1 - weight) * fc_players.loc[j][columns_to_avg]
C:\Users\mbolo\AppData\Local\Temp\ipykernel_10576\3833599976.py:16: SettingWithCopyWarning: 
A value is trying to be set on a 

In [24]:
fc_players = fc_players_newgk

fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions,vote_avg,vote_std
0,2428,P,Sommer,Inter,Sommer,,660,34,1988,0,...,22.9,27.9,395,18,4.6,14,0.41,12.1,6.117647,0.470588
1,133,P,Skorupski,Bologna,Skorupski,,658,32,1991,0,...,42.0,38.7,432,12,2.8,15,0.47,12.2,6.171875,0.461136
2,5876,P,Di Gregorio,Monza,Gregorio,,629,26,1997,0,...,52.5,44.2,469,19,4.1,18,0.56,11.5,6.348485,0.583678
3,572,P,Meret,Napoli,Meret,,639,26,1997,0,...,22.6,29.5,334,9,2.7,29,0.94,15.4,6.064516,0.435185
4,2814,P,Provedel,Lazio,Provedel,,652,29,1994,0,...,35.2,37.0,377,15,4.0,18,0.60,13.0,6.233333,0.478423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,6160,A,Vivaldo,Udinese,Vivaldo,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.212581,0.405030
538,6242,A,Bidaoui,Frosinone,Bidaoui,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.093504,0.362881
539,6418,A,Burnete,Lecce,Burnete,,87,19,2004,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.147232,0.532420
540,6419,A,Corfitzen,Lecce,Corfitzen,,137,18,2004,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.254964,0.341970


In [25]:
fc_players.to_excel('mid_outputs/season' + season + '/players_stats.xlsx')